In [1]:
from world_machine.train.scheduler import UniformScheduler, ChoiceScheduler
from world_machine.train.stages import StateSaveMethod

from world_machine_experiments.toy1d import Dimensions
from world_machine_experiments.shared.save_parameters import make_model

import hashlib


In [2]:
n_epoch = 100
n_variation = 0
configurations = {}
for n_segment in [2, 1]:
    fast_forward_choices = [False]
    if n_segment > 1:
        fast_forward_choices = [True, False]

    for fast_forward in fast_forward_choices:
        for stable_state_epochs in [1]:
            for check_input_masks in [True, False]:
                for state_save_method in [StateSaveMethod.MEAN, StateSaveMethod.REPLACE]:
                    for mask_sensorial_data in [UniformScheduler(0, 1, n_epoch)]:
                        for short_time_recall in [{Dimensions.MEASUREMENT}, set()]:

                            recall_n_past_choices = [0]
                            recall_stride_choices = [0]
                            if len(short_time_recall) > 0:
                                recall_n_past_choices = [5, 1, 0]
                                recall_stride_choices = [3, 1]

                            for recall_stride in recall_stride_choices:
                                for recall_n_past in recall_n_past_choices:

                                    recall_n_future_choices = []
                                    if len(short_time_recall) > 0 and recall_n_past > 0:
                                        recall_n_future_choices = [5, 1, 0]
                                    elif len(short_time_recall) > 0:
                                        recall_n_future_choices = [5, 1]

                                    for recall_n_future in recall_n_future_choices:
                                        for positional_encoder_type in ["alibi"]:
                                            for block_configuration in [[Dimensions.MEASUREMENT, Dimensions.MEASUREMENT],
                                                                        [Dimensions.MEASUREMENT, Dimensions.STATE_INPUT]]:
                                                for state_activation in ["tanh", None]:
                                                    state_regularizer = None
                                                    if state_activation is None:
                                                        state_regularizer = "mse"

                                                    for train_mse in [True]:
                                                        for train_stdw in [False]:
                                                            for noise_config in [None,
                                                                                    {"state": {
                                                                                        "mean": 0.0, "std": 0.1}},
                                                                                    {"measurement": {
                                                                                        "mean": 0.0, "std": 0.1}},
                                                                                    {"state": {"mean": 0.0, "std": 0.1}, "measurement": {"mean": 0.0, "std": 0.1}}]:

                                                                n_variation += 1

                                                                config = {"n_segment": n_segment,
                                                                            "fast_forward": fast_forward,
                                                                            "stable_state_epochs": stable_state_epochs,
                                                                            "check_input_masks": check_input_masks,
                                                                            "state_save_method": state_save_method,
                                                                            "mask_sensorial_data": mask_sensorial_data,
                                                                            "short_time_recall": short_time_recall,
                                                                            "recall_stride_past": recall_stride,
                                                                            "recall_stride_future": recall_stride,
                                                                            "recall_n_past": recall_n_past,
                                                                            "recall_n_future": recall_n_future,
                                                                            "positional_encoder_type": positional_encoder_type,
                                                                            "block_configuration": block_configuration,
                                                                            "state_activation": state_activation,
                                                                            "state_regularizer": state_regularizer,
                                                                            "train_mse": train_mse,
                                                                            "train_sdtw": train_stdw,
                                                                            "noise_config": noise_config}

                                                                model = make_model(
                                                                    config, "ParametersModel").model_validate(config)
                                                                model_json = model.model_dump_json()
                                                                variation_hash = int(hashlib.md5(model_json.encode('utf-8')).hexdigest(), 16)

                                                                configurations["variation"+str(
                                                                    variation_hash)] = config

#assert len(configurations) == n_variation

In [9]:
list(config.keys())

['n_segment',
 'fast_forward',
 'stable_state_epochs',
 'check_input_masks',
 'state_save_method',
 'mask_sensorial_data',
 'short_time_recall',
 'recall_stride_past',
 'recall_stride_future',
 'recall_n_past',
 'recall_n_future',
 'positional_encoder_type',
 'block_configuration',
 'state_activation',
 'state_regularizer',
 'train_mse',
 'train_sdtw',
 'noise_config']

In [6]:
n_variation

3072

In [5]:
assert len(configurations) == n_variation

In [3]:
configurations.keys()

dict_keys(['...'])

In [4]:
"variation117002763957487863391981534535771904748" in configurations

True

In [13]:
model_json = '{"n_segment":1,"fast_forward":false,"stable_state_epochs":1,"check_input_masks":true,"state_save_method":0,"mask_sensorial_data":"{\\"type\\": \\"UniformScheduler\\", \\"low_value\\": 0, \\"high_value\\": 1, \\"n_epoch\\": 100}","short_time_recall":[2],"recall_stride_past":1,"recall_stride_future":1,"recall_n_past":5,"recall_n_future":1,"positional_encoder_type":"alibi","block_configuration":[2,4],"state_activation":null,"state_regularizer":"mse","train_mse":true,"train_sdtw":false,"noise_config":{"measurement":{"mean":0.0,"std":0.1}}}'
variation_hash = int(hashlib.md5(model_json.encode('utf-8')).hexdigest(), 16)
variation_hash

97435974705371382585324094212583873313

In [14]:
variation_hash in configurations

False

In [15]:
"variation97435974705371382585324094212583873313" in configurations

True

In [3]:
n_variation

3072

In [4]:
26000/45

577.7777777777778

In [5]:
n_variation/577.7777777777778

5.316923076923077

In [7]:
(27.5*5.31)/24

6.084374999999999

In [9]:
from pydantic import BaseModel, ImportString, create_model
def _make_model(v, name):
    if type(v) is dict:
        return create_model(name, **{k: _make_model(v, k) for k, v in v.items()}), ...
    elif type(v) is type:
        return ImportString, v
    return type(v), v


def make_model(v: dict, name: str):
    return _make_model(v, name)[0]


def save_parameters(n_run: int,
                    base_seed: int,
                    output_dir: str,
                    parameters: dict[str]) -> dict:

    log = {"n_run": n_run,
           "base_seed": base_seed,
           "parameters": parameters}

    model: BaseModel = make_model(log, "ParametersModel").model_validate(log)
    model_json = model.model_dump_json(indent=4)

    file_path = os.path.join(output_dir, "parameters.json")

    with open(file_path, "w") as file:
        file.write(model_json)

    return {"path": file_path}

In [ ]:
n_epoch = 100

n_variation = 0
configurations = {}
for n_segment in [1, 2]:
    fast_forward_choices = [False]
    if n_segment > 1:
        fast_forward_choices = [True, False]

    for fast_forward in fast_forward_choices:
        for stable_state_epochs in [1]:
            for check_input_masks in [True, False]:
                for state_save_method in [StateSaveMethod.MEAN, StateSaveMethod.REPLACE]:
                    for mask_sensorial_data in [UniformScheduler(0, 1, n_epoch)]:
                        for short_time_recall in [set(), {Dimensions.measurement}]:

                            recall_n_past_choices = [0]
                            recall_stride_choices = [0]
                            if len(short_time_recall) > 0:
                                recall_n_past_choices = [0, 1, 5]
                                recall_stride_choices = [1, 3]

                            for recall_stride in recall_stride_choices:
                                for recall_n_past in recall_n_past_choices:

                                    recall_n_future_choices = []
                                    if len(short_time_recall) > 0 and recall_n_past > 0:
                                        recall_n_future_choices = [0, 1, 5]
                                    elif len(short_time_recall) > 0:
                                        recall_n_future_choices = [1, 5]

                                    for recall_n_future in recall_n_future_choices:
                                        for positional_encoder_type in ["alibi"]:
                                            for block_configuration in [[Dimensions.measurement, Dimensions.measurement],
                                                                        [Dimensions.measurement, Dimensions.STATE_INPUT]]:
                                                for state_activation in ["tanh", "ltanh", None]:
                                                    state_regularizer = None
                                                    if state_activation is None:
                                                        state_regularizer = "mse"

                                                    n_variation += 1

                                                    config = {"n_segment": n_segment,
                                                            "fast_forward": fast_forward,
                                                            "stable_state_epochs":stable_state_epochs,
                                                            "check_input_masks": check_input_masks,
                                                            "state_save_method": state_save_method,
                                                            "mask_sensorial_data": mask_sensorial_data,
                                                            "short_time_recall": short_time_recall,
                                                            "recall_stride_past": recall_stride,
                                                            "recall_stride_future": recall_stride,
                                                            "recall_n_past": recall_n_past,
                                                            "recall_n_future": recall_n_future,
                                                            "positional_encoder_type":positional_encoder_type,
                                                            "block_configuration": block_configuration,
                                                            "state_activation": state_activation}

                                                    model: BaseModel = make_model(config, "ParametersModel").model_validate(config)
                                                    model_json = model.model_dump_json(indent=4)
                                                    variation_hash = hash(model_json)

                                                    configurations[variation_hash] = config
                                                    
                                                    

assert len(configurations) == n_variation
n_variation, n_variation/800, (n_variation/800)**0.5

In [23]:
len(configurations)

1152

In [20]:
import numpy as np

_, counts = np.unique(hist, return_counts=True)

counts.max()

1

In [8]:
codes[0]

'1FalseTrueMEAN<world_machine.train.scheduler.UniformScheduler object at 0x00000198292BAC00>{<Dimensions.measurement: 2>}101[<Dimensions.measurement: 2>, <Dimensions.measurement: 2>]tanh'

In [59]:
(30*7*24*60*60)/22757.4

797.2791267895277

In [60]:
n_variation/((30*7*24*60*60)/22757.1333333)

11.559178835961905

In [45]:
with open("test.bin", "wb") as file:
    pickle.dump(configurations, file)

In [46]:
with open("test.bin", "rb") as file:
    configurations2 = pickle.load(file)

In [48]:
configurations2[0]

(1,
 False,
 1,
 True,
 <StateSaveMethod.MEAN: 1>,
 0.5,
 (<Dimensions.measurement: 2>,),
 1,
 0,
 1,
 'alibi',
 (),
 (<Dimensions.measurement: 2>, <Dimensions.measurement: 2>),
 'tanh')